In [1]:
import numpy as np
import nibabel as nib
from nibabel.affines import apply_affine
from nibabel.streamlines import Tractogram, TckFile 
from fury import window, actor

def cargar_datos(eigenvector_path, mask_path):
    """
    Carga los datos de eigenvectores y de la máscara binaria.
    
    Parámetros:
      eigenvector_path: Ruta al archivo de eigenvectores (NIfTI).
      mask_path: Ruta al archivo de la máscara binaria (NIfTI).
      
    Retorna:
      eigen_data: Arreglo numpy de eigenvectores con forma (X,Y,Z,3).
      mask_data: Arreglo numpy binario con forma (X,Y,Z).
      affine: Matriz de transformación del espacio imagen.
    """
    eigen_img = nib.load(eigenvector_path)
    eigen_data = eigen_img.get_fdata()
    mask_img = nib.load(mask_path)
    mask_data = mask_img.get_fdata()
    affine = eigen_img.affine  # se asume que ambos archivos comparten la misma transformación
    # print(eigen_data.shape)
    # print(mask_data.shape)
    # print(eigen_data)
    # print(mask_data)
    return eigen_data, mask_data, affine

def tracking_streamline(eigen_data, mask, affine, seed, step_size=1.0, max_steps=1000):
    """
    Reconstruye un streamline partiendo del punto semilla y en cada paso se obtiene el eigenvector principal
    en el voxel correspondiente. Se verifica que el nuevo punto se encuentre dentro de
    la máscara y de los límites de la imagen.
    
    Parámetros:
      eigen_data: Arreglo con los eigenvectores 
      mask: Arreglo binario (X,Y,Z) que indica las regiones válidas.
      affine: Matriz de transformación 
      seed: Coordenadas del punto semilla 
      step_size: Tamaño del paso en unidades espaciales.
      max_steps: Número máximo de iteraciones.
      
    Retorna:
      streamline: Matriz numpy de puntos del streamline, de forma (N, 3).
    """
    streamline = [np.array(seed)]
    current_point = np.array(seed)
    inv_affine = np.linalg.inv(affine)
    
    for i in range(max_steps):
        current_point = np.asarray(current_point).flatten()[:3]
        current_point_reshaped = current_point.reshape(1, 3)
        
        voxel_index = np.round(apply_affine(inv_affine, current_point_reshaped)).astype(int).squeeze()
               
        # Verificar si almenos un valor es negativo o se encuentra fuera de los limites de la mascara
        if (voxel_index < 0).any() or (voxel_index >= np.array(mask.shape)).any():
            print("Punto fuera de los límites.")
            break
        
        # Extraemos el primer eigenvector 
        # (verificar valores)
        
        # extraccion_vecindad
        
        vec_full = eigen_data[voxel_index[0], voxel_index[1], voxel_index[2], :]
        vec = vec_full[:1]  # utilizamos las primeras 3 componentes
        
        print(vec)
        
        norm_vec = np.linalg.norm(vec)
        
        # Normalizacion
        direction = vec / norm_vec
        #AQUI VA LA RED 
        
        # Corroborar que el actual y el nuevo sean paralelos (no antiparalelos) via producto punto >0

        
        # Prediccion
        new_point = current_point + step_size * direction
        
        streamline.append(new_point)
        current_point = new_point

    # Usamos vstack para asegurar regresar un arreglo numerico y evitar problemas de inhomogeneidad
    return np.vstack(streamline)




def guardar_trk(streamline, affine, output_file):
    """
    Guarda el streamline en un archivo .trk.
    
    Parámetros:
      streamline: Arreglo numpy con la trayectoria (N x 3).
      affine: Matriz de transformación (voxel a RASmm).
      output_file: Ruta del archivo de salida .trk.
    """
    # Se crea un tractograma con el streamline
    tractogram = Tractogram([streamline], affine_to_rasmm=affine)
    # trk_obj = TrkFile(tractogram, header={})
    tck_obj = TckFile(tractogram, header={})
    # nib.streamlines.save(trk_obj, output_file)
    nib.streamlines.save(tck_obj, output_file)
    print(f"Streamline guardado como: {output_file}")

def visualizar_streamline(streamline):
    """
    Visualiza en 3D el streamline utilizando fury.
    
    Parámetros:
      streamline: Arreglo numpy de puntos del streamline.
    """
    scene = window.Scene()
    # Se crea un actor de línea (recibe una lista de streamlines, cada uno como un array Nx3)
    line_actor = actor.line([streamline], colors=(1, 0, 0))
    scene.add(line_actor)
    window.show(scene)


In [12]:
# inputs
eigenvector_file = 'tensor_evecs_stanford.nii.gz'
mask_file = 'mask_stanford.nii.gz'                
seed_point = [10, 40, 10]                 
    
eigen_data, mask_data, affine = cargar_datos(eigenvector_file, mask_file)
    
# Reconstrucción del streamline 
streamline = tracking_streamline(eigen_data, mask_data, affine, seed_point, step_size=1, max_steps=100)
    

output_trk = 'streamline_unidir.tck' # cambiar a .tck
guardar_trk(streamline, affine, output_trk)
    
visualizar_streamline(streamline)


[[ 0.02885396  0.72462761 -0.68853635]]
[[ 0.02885396  0.72462761 -0.68853635]]
[[0.14533328 0.4265762  0.89269871]]
[[-0.17974198  0.22764525  0.95701122]]
[[-0.17974198  0.22764525  0.95701122]]
[[ 0.0397147  -0.41696247  0.90805566]]
[[ 0.0397147  -0.41696247  0.90805566]]
[[-0.49015376  0.85045904 -0.19096793]]
[[-0.49015376  0.85045904 -0.19096793]]
[[-0.41224968  0.44097197  0.79724145]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
[[0.39210916 0.87851089 0.27289021]]
Punto fuera de los límites.
Streamline guardado como: streamline_unidir.tck


- Pasarlo a tck y graficarlo con la imagen original de fondo
- Completar el streamline en la direccion opuesta
- Verificar via gráfica con orientaciones y streamline
- Presentacion tesis con pseudocódigo y la aplicacion de la red neuronal (primera semana de abril)